## 1. Dependencias y variables

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
shell_output = !gcloud auth list 2>/dev/null
SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()
print("Service Account:", SERVICE_ACCOUNT)

Service Account: 330930586045-compute@developer.gserviceaccount.com


In [ ]:
USER_FLAG = "--user"
!pip3 install {USER_FLAG} google-cloud-aiplatform --upgrade
!pip3 install {USER_FLAG} kfp==2.4.0 google-cloud-pipeline-components
!pip3 install {USER_FLAG} gcsfs
!pip install -U google-cloud-aiplatform "shapely>2"

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)


In [ ]:
!python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
!python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

In [ ]:
import os
PROJECT_ID = ""
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)


## 2. Librerias

In [3]:
import os
import pprint as pp
import sys

import pickle
import os
import argparse

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error,mean_absolute_percentage_error
import numpy as np
import pandas as pd

from google.cloud import storage
from google.cloud import aiplatform 

## 3. Modelo

In [4]:
BUCKET_NAME="gs://bucket2025nahumfg/inputs/trabajo_final_tercer_estadio"
url = BUCKET_NAME+"/CreditScoring.csv"
data = pd.read_csv(url)
data.head()

,ID,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [5]:
modelling_columns = [
                      'SeriousDlqin2yrs'
                    , 'RevolvingUtilizationOfUnsecuredLines'
                    , 'age'
                    , 'NumberOfTime30-59DaysPastDueNotWorse'
                    , 'DebtRatio'
                    , 'MonthlyIncome'
                    , 'NumberOfOpenCreditLinesAndLoans'
                    , 'NumberOfTimes90DaysLate'
                    ,'NumberRealEstateLoansOrLines'
                    , 'NumberOfTime60-89DaysPastDueNotWorse'
                    ,'NumberOfDependents'
                    ]
data = data[modelling_columns]

In [6]:
train_size = 0.8
test_size = 0.1
valid_size = 0.1

train_ds, valid_ds, test_ds = np.split(data.sample(frac=1, random_state=42), [int((train_size)*len(data)), int((1-test_size)*len(data))])

In [7]:
target = "SeriousDlqin2yrs"

x_train = train_ds.drop(columns=target, axis=1)
y_train = train_ds[target]

x_valid = valid_ds.drop(columns=target, axis=1)
y_valid = valid_ds[target]

x_test = test_ds.drop(columns=target, axis=1)
y_test = test_ds[target]


In [8]:
model = RandomForestRegressor()
model.fit(x_train , y_train)

RandomForestRegressor()

In [9]:
y_pred = model.predict(x_valid)

#evaluate Model
adj_r2 = r2_score(y_true=y_valid, y_pred=y_pred)
mae = mean_absolute_error(y_true=y_valid, y_pred=y_pred)
mse = mean_squared_error(y_true=y_valid, y_pred=y_pred)
mape = mean_absolute_percentage_error(y_true=y_valid, y_pred=y_pred)
rmse = np.sqrt(mse)
print(f"Adjusted R2 : {adj_r2}")
print(f"Mean Absolute Error : {mae}")
print(f"Mean Absolute Percentage Error : {round(mape,4)*100}%")
print(f"Mean Squared Error : {mse}")
print(f"Root Mean Squared Error : {rmse}")

Adjusted R2 : 0.1753158620137517
Mean Absolute Error : 0.10277432922223403
Mean Absolute Percentage Error : 2.412625609660365e+16%
Mean Squared Error : 0.05278725830204145
Root Mean Squared Error : 0.2297547786272169


## 4. Primera forma de desplegar

In [10]:
MODEL_PATH=BUCKET_NAME+"/models/"
model_path = "./" + "model.pkl"
with open(model_path, 'wb') as file:  
    pickle.dump(model, file) 
    
#copy model artifacts to GCS storage
!gsutil cp "model.pkl" $MODEL_PATH

Copying file://model.pkl [Content-Type=application/octet-stream]...
\ [1 files][ 81.7 MiB/ 81.7 MiB]                                                
Operation completed over 1 objects/81.7 MiB.                                     


In [11]:
#Prediction containers list available at : https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers
serving_container_uri = "us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-3:latest"

#define GCS location for model artifacts
artifact_uri = MODEL_PATH

#Upload Model to Vertex AI Model Registry using Python SDK
model = aiplatform.Model.upload(display_name= "MLOps0-model" ,
                                    artifact_uri=artifact_uri,
                                    serving_container_image_uri=serving_container_uri)


Creating Model
Create Model backing LRO: projects/330930586045/locations/us-central1/models/5677153467672559616/operations/1965766158686617600
Model created. Resource name: projects/330930586045/locations/us-central1/models/5677153467672559616@1
To use this Model in another session:
model = aiplatform.Model('projects/330930586045/locations/us-central1/models/5677153467672559616@1')


In [12]:
#Create the model endpoint using Python SDK
endpoint = model.deploy(machine_type="n1-standard-4",
                        min_replica_count=1,
                        max_replica_count=1)

Creating Endpoint
Create Endpoint backing LRO: projects/330930586045/locations/us-central1/endpoints/6830475194511917056/operations/2094224301182943232
Endpoint created. Resource name: projects/330930586045/locations/us-central1/endpoints/6830475194511917056
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/330930586045/locations/us-central1/endpoints/6830475194511917056')
Deploying model to Endpoint : projects/330930586045/locations/us-central1/endpoints/6830475194511917056
Deploy Endpoint model backing LRO: projects/330930586045/locations/us-central1/endpoints/6830475194511917056/operations/679988465072340992
Endpoint model deployed. Resource name: projects/330930586045/locations/us-central1/endpoints/6830475194511917056


In [14]:
#Test the model endpoint using Python SDK

#create list to hold request data
instances = [
    [0.766127, 45, 2, 0.802982, 9120.0, 13, 0, 6, 0, 2.0],
  ]

prediction = endpoint.predict(instances=instances)

print(prediction)


Prediction(predictions=[0.75], deployed_model_id='2572906687019089920', metadata=None, model_version_id='1', model_resource_name='projects/330930586045/locations/us-central1/models/5677153467672559616', explanations=None)


In [16]:
# Undeploy the model and delete the endpoint
endpoint.undeploy_all()
endpoint.delete()
model.delete()

Undeploying Endpoint model: projects/330930586045/locations/us-central1/endpoints/6830475194511917056
Undeploy Endpoint model backing LRO: projects/330930586045/locations/us-central1/endpoints/6830475194511917056/operations/2526569865410510848
Endpoint model undeployed. Resource name: projects/330930586045/locations/us-central1/endpoints/6830475194511917056
Deleting Endpoint : projects/330930586045/locations/us-central1/endpoints/6830475194511917056
Endpoint deleted. . Resource name: projects/330930586045/locations/us-central1/endpoints/6830475194511917056
Deleting Endpoint resource: projects/330930586045/locations/us-central1/endpoints/6830475194511917056
Delete Endpoint backing LRO: projects/330930586045/locations/us-central1/operations/7138255883837898752
Endpoint resource projects/330930586045/locations/us-central1/endpoints/6830475194511917056 deleted.
Deleting Model : projects/330930586045/locations/us-central1/models/5677153467672559616
Model deleted. . Resource name: projects/3

## 5. Segunda forma de desplegar

In [18]:
from typing import NamedTuple

import kfp
from kfp import dsl
from kfp.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,
                        OutputPath, component , ClassificationMetrics , Metrics)

from kfp import compiler
from kfp.components import load_component_from_file

import json
import yaml

In [21]:
!mkdir components

In [22]:
@component(
    packages_to_install=["pandas", "pyarrow" , "fsspec" , "gcsfs"],
    base_image="python:3.9",
    output_component_file="./components/download_data.yaml"
)
def download_data(input_data_path : str
                  , input_data_filename : str
                  , downloaded_data : Output[Dataset]):
    
    import pandas as pd
    import os
    
    print(f"input_data_path : {input_data_path}")
    print(f"input_data_filename : {input_data_filename}")
    print(f"downloaded_data : {downloaded_data}")
    print(f"downloaded_data.path : {downloaded_data.path}")
                
    url = os.path.join(input_data_path , input_data_filename)
    
    #read data from GCS location
    data = pd.read_csv(url)
    
    #write to output dataset path
    output_data_uri = downloaded_data.path + ".csv" 
    data.to_csv(output_data_uri 
                , index=False
                , encoding='utf-8-sig')

In [23]:
@component(
    packages_to_install=["pandas", "pyarrow", "fsspec" , "gcsfs"],
    base_image="python:3.9",
    output_component_file="./components/preprocess_data.yaml"
)
def preprocess_data(train_size : float 
                    , test_size : float
                    , valid_size : float
                    , train_data : Output[Dataset]
                    , valid_data : Output[Dataset]
                    , test_data : Output[Dataset]
                    , input_data:  Input[Dataset]
                   ):
    
    import numpy as np
    import pandas as pd
    
    print(f"train_size : {train_size}")
    print(f"test_size : {test_size}")
    print(f"valid_size : {valid_size}")
    print(f"train_data : {train_data}")
    print(f"valid_data : {valid_data}")
    print(f"test_data : {test_data}")
    print(f"input_data : {input_data}")
    
    data = pd.read_csv(input_data.path + ".csv")
    
    modelling_columns = [   
                      'SeriousDlqin2yrs'
                    , 'RevolvingUtilizationOfUnsecuredLines'
                    , 'age'
                    , 'NumberOfTime30-59DaysPastDueNotWorse'
                    , 'DebtRatio'
                    , 'MonthlyIncome'
                    , 'NumberOfOpenCreditLinesAndLoans'
                    , 'NumberOfTimes90DaysLate'
                    ,'NumberRealEstateLoansOrLines'
                    , 'NumberOfTime60-89DaysPastDueNotWorse'
                    ,'NumberOfDependents'
                    ]
    
    data = data[modelling_columns]

    train_ds, valid_ds, test_ds = np.split(data.sample(frac=1, random_state=42), [int((train_size)*len(data)), int((1-test_size)*len(data))])
    train_ds.to_csv(train_data.path + ".csv" , index=False, encoding='utf-8-sig')
    valid_ds.to_csv(valid_data.path + ".csv" , index=False, encoding='utf-8-sig')
    test_ds.to_csv(test_data.path + ".csv" , index=False, encoding='utf-8-sig')


In [27]:
print(train_ds.shape)
print(valid_ds.shape)
print(test_ds.shape)

(120000, 11)
(15000, 11)
(15000, 11)


In [28]:
@component(
    packages_to_install=["kfp==2.4.0", "pandas", "pyarrow",  "scikit-learn==1.3.2" , "fsspec" , "gcsfs", "click==8.1.7", "docstring-parser==0.16", "kfp-pipeline-spec==0.2.2", "kfp-server-api==2.0.5", "kubernetes==26.1.0", "PyYAML==6.0.2", "requests-toolbelt==0.10.1", "tabulate==0.9.0", "protobuf==3.20.3", "urllib3==1.26.20"]
    , base_image="python:3.9"
    , output_component_file="./components/train.yaml"
)
def train_model(
    train_data:  Input[Dataset],
    model: Output[Model], 
):
    
    print(f"train_data : {train_data}")
    print(f"model : {model}")
    
    from sklearn.ensemble import RandomForestRegressor
    import pandas as pd
    import pickle
    import sklearn

    train_ds = pd.read_csv(train_data.path+".csv")
    my_model = RandomForestRegressor()
    
    target = "cnt"
    
    x_train = train_ds.drop(columns=target, axis=1)
    y_train = train_ds[target]
    
    my_model.fit(x_train , y_train)
    model.metadata["model_name"] = "RandomForestRegressor"
    model.metadata["framework"] = "sklearn"
    model.metadata["framework_version"] = sklearn.__version__
    file_name = model.path + f".pkl"
    
    with open(file_name, 'wb') as file:  
        pickle.dump(my_model, file)


In [29]:
@component(
    packages_to_install=["pandas", "pyarrow",  "scikit-learn==1.3.2" , "fsspec" , "gcsfs"]
    , base_image="python:3.9"
    , output_component_file="./components/evaluate_model.yaml"
)
def evaluate_model(
    test_data:  Input[Dataset],
    model: Input[Model], 
    target_column_name : str ,
    deployment_metric : str ,
    deployment_metric_threshold : float ,
    kpi: Output[Metrics]
)-> NamedTuple(
    "Outputs",
    [
        ("deploy_flag", str),  # Return parameter.
    ],
):
    
    print(f"test_data : {test_data}")
    print(f"model : {model}")
    print(f"kpi : {kpi}")
    print(f"deployment_metric : {deployment_metric}")
    print(f"deployment_metric_threshold : {deployment_metric_threshold}")
    
    from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error,mean_absolute_percentage_error
    import pandas as pd
    import pickle
    import numpy as np
    import json
    
    test_ds = pd.read_csv(test_data.path+".csv")
    target = target_column_name
    
    x_test = test_ds.drop(columns=target, axis=1)
    y_test = test_ds[target]
    
    print(f"model.path : {model.path}")
    file_name = model.path + f".pkl"
    print(f"file_name : {file_name}")
    #model = pickle.loads(file_name)
    with open(file_name, 'rb') as file:  
        model = pickle.load(file)
    
    y_pred = model.predict(x_test)
    r2 = r2_score(y_true=y_test, y_pred=y_pred)
    mae = mean_absolute_error(y_true=y_test, y_pred=y_pred)
    mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
    mape = mean_absolute_percentage_error(y_true=y_test, y_pred=y_pred)
    rmse = np.sqrt(mse)
    
    model_metrics = {"r2" : r2 
                     , "mae" : mae 
                     , "mape" : mape 
                     , "mse" : mse 
                     , "rmse" : rmse
                    }
    
    print(f"Adjusted_R2 : {r2}")
    print(f"Mean Absolute Error : {mae}")
    print(f"Mean Absolute Percentage Error : {round(mape,4)*100}%")
    print(f"Mean Squared Error : {mse}")
    print(f"Root Mean Squared Error : {rmse}")
    
    kpi.log_metric("Adjusted_R2", float(r2))
    kpi.log_metric("Mean Absolute Error", float(mae))
    kpi.log_metric("Mean Absolute Percentage Error", float(mape))
    kpi.log_metric("Mean Squared Error", float(mse))
    kpi.log_metric("Root Mean Squared Error", float(rmse))
    
    actual_metric_value = model_metrics.get(deployment_metric)
    
    if actual_metric_value >= deployment_metric_threshold:
        deploy_flag = "True"
    else:
        deploy_flag = "False"
        
    return (deploy_flag,)


In [30]:
@component(
    packages_to_install=["pandas", "pyarrow",  "scikit-learn==1.3.2" , "fsspec" , "gcsfs" , "google-cloud-aiplatform"]
    , base_image="python:3.9"
    , output_component_file="./components/register_model.yaml"
)
def register_model(
    serving_container_uri : str ,
    project_id : str ,
    region: str,
    model_name : str , 
    model: Input[Model], 
)-> NamedTuple(
    "Outputs",
    [
        ("model_resource_name", str),  # Return parameter.
    ],
):
    
    print(f"serving_container_uri : {serving_container_uri}")
    print(f"project_id : {project_id}")
    print(f"region : {region}")
    print(f"model : {model}")
    
    from google.cloud import aiplatform
    
    print(f"model.uri : {model.uri[:-5]}")
    
    aiplatform.init(project = project_id , location=region)
    model = aiplatform.Model.upload(display_name= model_name ,
                                    artifact_uri=model.uri[:-5],
                                    serving_container_image_uri=serving_container_uri)
    return (model.resource_name,)

In [31]:
@component(
    packages_to_install=["kfp==2.4.0", "pandas", "pyarrow",  "scikit-learn==1.3.2" , "fsspec" , "gcsfs", "google-cloud-aiplatform", "click==8.1.7", "kfp-pipeline-spec==0.2.2", "kfp-server-api==2.0.5", "kubernetes==26.1.0", "PyYAML==6.0.2", "requests-toolbelt==0.10.1", "tabulate==0.9.0", "protobuf==3.20.3", "urllib3==1.26.20", "numpy==1.24.4", "google-cloud-pipeline-components==2.8.0"]
    , base_image="python:3.9"
    , output_component_file="./components/deploy_model.yaml"
)
def deploy_model(
    model_resource_name : str ,
    project_id : str ,
    region: str
)-> NamedTuple(
    "Outputs",
    [
        ("endpoint_resource_name", str),  # Return parameter.
    ],
):
    
    print(f"model_resource_name : {model_resource_name}")
    print(f"project_id : {project_id}")
    print(f"region : {region}")
    
    from google.cloud import aiplatform
    
    aiplatform.init(project = project_id , location=region)
    
    model = aiplatform.Model(model_resource_name)
    endpoint = model.deploy(machine_type="n1-standard-4",
                        min_replica_count=1,
                        max_replica_count=1)
    
    return (endpoint.resource_name,)


In [33]:
PROJECT_ID = 'wired-caldron-412419'

In [34]:
shell_output = !gcloud auth list 2>/dev/null
SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()
print("Service Account:", SERVICE_ACCOUNT)
print("Project ID: ", PROJECT_ID)
print("staging_bucket_uri: ",BUCKET_NAME)
print("input_data_path: ",BUCKET_NAME)


Service Account: 330930586045-compute@developer.gserviceaccount.com
Project ID:  wired-caldron-412419
staging_bucket_uri:  gs://bucket2025nahumfg/inputs/trabajo_final_tercer_estadio
input_data_path:  gs://bucket2025nahumfg/inputs/trabajo_final_tercer_estadio


In [35]:
%%writefile config.json
{
    "project":"wired-caldron-412419",
    "region":"us-central1",
    "service_account":"330930586045-compute@developer.gserviceaccount.com",
    "staging_bucket_uri":"gs://bucket2025nahumfg/inputs/trabajo_final_tercer_estadio",
    "pipeline_name":"tabular-data-regression-kfp-cicd-pipeline",
    "pipeline_package_path":"tabular-data-regression-kfp-cicd-pipeline.json",
    "input_data_path":"gs://bucket2025nahumfg/inputs/trabajo_final_tercer_estadio",
    "input_data_filename":"CreditScoring.csv",
    "target_column_name":"SeriousDlqin2yrs",
    "train_size":0.8,
    "test_size":0.1,
    "valid_size":0.1,
    "deployment_metric":"r2",
    "deployment_metric_threshold":0.8,
    "model_name":"model_tabular_regression",
    "serving_container_uri":"us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-3:latest"
}

Writing config.json


In [36]:
%%writefile build_pipeline.py

import json
import yaml

import kfp
from kfp import dsl
from kfp import compiler
from kfp.components import load_component_from_file

download_data = load_component_from_file("./components/download_data.yaml")
preprocess_data = load_component_from_file("./components/preprocess_data.yaml")
train_model = load_component_from_file("./components/train.yaml")
evaluate_model = load_component_from_file("./components/evaluate_model.yaml")
register_model = load_component_from_file("./components/register_model.yaml")
deploy_model = load_component_from_file("./components/deploy_model.yaml")

#read configuration from file
with open("config.json") as json_file:
    config = json.load(json_file)
    
PIPELINE_NAME = config.get("pipeline_name")
PACKAGE_PATH = config.get("pipeline_package_path")
BUCKET_URI = config.get("staging_bucket_uri")
PIPELINE_ROOT = "{}/pipeline_root/kfp_tabular_data_regression".format(BUCKET_URI)
print(f"PIPELINE_ROOT :{PIPELINE_ROOT}")

@dsl.pipeline(
    # Default pipeline root. You can override it when submitting the pipeline.
    pipeline_root=PIPELINE_ROOT,
    # A name for the pipeline. Use to define the pipeline Context.
    name=PIPELINE_NAME,
    
)
def pipeline(project: str = "",
 region: str = "",
 service_account: str = "",
 staging_bucket_uri: str = "",
 pipeline_name: str = "",
 pipeline_package_path: str = "",
 input_data_path: str = "",
 input_data_filename: str = "",
 target_column_name: str = "",
 train_size: float = 0.8,
 test_size: float = 0.1,
 valid_size: float = 0.1,
 hypertune_container_image_uri: str = "" ,
 hypertune_machine_type: str = "",
 hypertune_machine_replica_count: int = 1 ,
 hypertune_max_trial_count: int = 1 ,
 hypertune_parallel_trial_count: int = 1 ,
 hypertune_metric : str = "" ,
 hypertune_metric_objective : str = "" ,
 hypertune_job_name: str = "" ,
 deployment_metric: str = "" ,
 deployment_metric_threshold: float = 0.8 ,
 serving_container_uri : str = "" ,
 model_name : str = "", 
 user_email : str = "",
 monitoring_job_name : str = "" ,
 predict_instance_schema_uri : str = ""
):
    
    download_data_op = download_data(input_data_path = input_data_path 
                                     , input_data_filename = input_data_filename
                                    )
    
    
    preprocess_data_op = preprocess_data(train_size = train_size
                                         , test_size = test_size
                                         , valid_size = valid_size
                                         , input_data = download_data_op.outputs["downloaded_data"])
    
    train_model_op = train_model(train_data = preprocess_data_op.outputs["train_data"])
    
    
    evaluate_model_op = evaluate_model(test_data = preprocess_data_op.outputs["test_data"]
                                       ,model = train_model_op.outputs["model"]
                                       ,target_column_name = target_column_name
                                       ,deployment_metric = deployment_metric 
                                       ,deployment_metric_threshold = deployment_metric_threshold
                                      )
    
    with dsl.If(evaluate_model_op.outputs["deploy_flag"] == "True"):
        
        register_model_op = register_model(serving_container_uri = serving_container_uri 
                                       , model = train_model_op.outputs["model"]
                                       , model_name = model_name
                                       , project_id = project 
                                       , region = region)
        
        #deploy only if metric value exceeds deployment threshold
        deploy_model_op = deploy_model(model_resource_name = register_model_op.outputs["model_resource_name"] 
                                   , project_id = project 
                                   , region = region)
    
compiler.Compiler().compile(
    pipeline_func=pipeline
    , package_path=PACKAGE_PATH
)


Writing build_pipeline.py


In [37]:
%%writefile run_pipeline.py

from google.cloud import aiplatform
import yaml
import json

with open("config.json") as json_file:
    config = json.load(json_file)
    
SERVICE_ACCOUNT = config.get("service_account")
DISPLAY_NAME = config.get("pipeline_name")
PACKAGE_PATH = config.get("pipeline_package_path")
BUCKET_URI = config.get("staging_bucket_uri")
PIPELINE_ROOT = "{}/pipeline_root/kfp_tabular_data_regression".format(BUCKET_URI)
print(f"PIPELINE_ROOT :{PIPELINE_ROOT}")

job = aiplatform.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path=PACKAGE_PATH,
    pipeline_root=PIPELINE_ROOT,
    parameter_values=config,
)

job.submit(service_account = SERVICE_ACCOUNT)


Writing run_pipeline.py


In [38]:
!python3 build_pipeline.py

PIPELINE_ROOT :gs://bucket2025nahumfg/inputs/trabajo_final_tercer_estadio/pipeline_root/kfp_tabular_data_regression


In [39]:
!python3 run_pipeline.py

PIPELINE_ROOT :gs://bucket2025nahumfg/inputs/trabajo_final_tercer_estadio/pipeline_root/kfp_tabular_data_regression
Creating PipelineJob
PipelineJob created. Resource name: projects/330930586045/locations/us-central1/pipelineJobs/tabular-data-regression-kfp-cicd-pipeline-20250208040506
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/330930586045/locations/us-central1/pipelineJobs/tabular-data-regression-kfp-cicd-pipeline-20250208040506')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/tabular-data-regression-kfp-cicd-pipeline-20250208040506?project=330930586045


In [61]:
! gsutil cp ./config.json $BUCKET_NAME
! gsutil cp ./tabular-data-regression-kfp-cicd-pipeline.json $BUCKET_NAME
! gsutil acl ch -u AllUsers:R $BUCKET_NAME/config.json
! gsutil acl ch -u AllUsers:R $BUCKET_NAME/tabular-data-regression-kfp-cicd-pipeline.json

Copying file://./config.json [Content-Type=application/json]...
/ [1 files][  823.0 B/  823.0 B]                                                
Operation completed over 1 objects/823.0 B.                                      
Copying file://./tabular-data-regression-kfp-cicd-pipeline.json [Content-Type=application/json]...
/ [1 files][ 31.6 KiB/ 31.6 KiB]                                                
Operation completed over 1 objects/31.6 KiB.                                     
